In [7]:
#!/usr/bin/env python
# coding: utf-8

import json, math, copy
from geosnap.io import store_ltdb
from geosnap import Community, datasets
from geosnap.io import store_census
import pandas as pd
import shapely.wkt
import shapely.geometry
from datetime import datetime
from datetime import timedelta
from pathlib import Path
import urllib.parse
import webbrowser
import os
import pprint
import numpy as np
from notebook import notebookapp
from IPython.core.display import display, HTML
import geopandas as gpd

param = {
        'title': "Adaptive Choropleth Mapper with Correlogram",
            'filename_suffix': "SD_correlogram",
            'state_fips': None,
            'msa_fips': "41740",  #For more options: http://osnap.cloud/~suhan/LNE/pick_POI.html
            'county_fips': None,
            'year': 2000,
            'NumOfMaps': 6,
            'variables': [
                "p_other_language",
                "p_female_headed_families",
                "median_income_blackhh",
                "median_income_hispanichh",
                "median_income_asianhh",
                "per_capita_income",      
            ],
            'chart': "Correlogram",
            'label': "short_name", 
}       

# variable, short_name or full_name# convert year, variable to years, variables in the param
if ('years' not in param and 'year' in param): param['years'] = [param['year']]
if ('variables' not in param and 'variable' in param): param['variables'] = [param['variable']]
#print(param)

# select community by state_fips, msa_fips, county_fips
community = None
if ('msa_fips' in param and param['msa_fips']):
    community = Community.from_ltdb(years=param['years'], msa_fips=param['msa_fips'])
    #community = Community.from_ltdb(msa_fips=param['msa_fips'])
elif ('county_fips' in param and param['county_fips']):
    community = Community.from_ltdb(years=param['years'], county_fips=param['county_fips'])
elif ('state_fips' in param and param['state_fips']):
    community = Community.from_ltdb(years=param['years'], state_fips=param['state_fips'])
#print(community.gdf)


codebook = pd.read_csv('template/conversion_table_codebook.csv')
codebook.set_index(keys='variable', inplace=True)
labels = copy.deepcopy(param['variables'])
label = 'short_name'                                             # default
if (param['label'] == 'variable'): label = 'variable'
if (param['label'] == 'full_name'): label = 'full_name'
if (param['label'] == 'short_name'): label = 'short_name'
if (label != 'variable'):
    for idx, variable in enumerate(param['variables']):
        try:
            codeRec = codebook.loc[variable]
            labels[idx] = codeRec[label]
        except:
            print("variable not found in codebook.  variable:", variable)
param['labels'] = labels
    
community.gdf
community.gdf.to_csv('output_temp.csv')